In [23]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

Import Train Data

In [ ]:
# Import
root_path = "C:/QRT 2025 Reconstruction of Liquid Asset Performance/Data"

x_train_path = root_path + "/X_train_itDkypA.csv"
y_train_path = root_path + "/y_train_3LeeT2g.csv"

x_train = pd.read_csv(x_train_path)
y_train = pd.read_csv(y_train_path)

x_train.fillna(0, inplace=True)
y_train.fillna(0, inplace=True)

,ID,RET_TARGET
0,0,-0.022351
1,1,-0.011892
2,2,-0.015285
3,3,-0.019226
4,4,0.006644
...,...,...
267095,267095,0.002080
267096,267096,-0.002565
267097,267097,-0.018406
267098,267098,0.045101


In [25]:
# Consolidate Data in to a single DataFrame : All_data
All_data = x_train.copy()
All_data['RET_TARGET'] = y_train['RET_TARGET']

# List all columns in All_data
All_data_columns = All_data.columns.tolist()
#print("All_data columns:", All_data_columns)

# Obtain the unique values of the 'ID_TARGET' column
unique_ids = All_data['ID_TARGET'].unique().tolist()
unique_ids.sort() # Sort the unique IDs for better readability
print("Unique IDs in All_data:", unique_ids,'\n',len(unique_ids)," unique IDs found.")

# Example
# Filter the DataFrame by a specific ID_TARGET
id_to_filter = unique_ids[15]  # Change this to the desired ID_TARGET
filtered_data = All_data[All_data['ID_TARGET'] == id_to_filter]
print("ID_TARGET :",id_to_filter,"with rows",len(filtered_data))

Unique IDs in All_data: [1, 3, 7, 8, 9, 13, 16, 17, 19, 21, 22, 25, 28, 29, 33, 34, 38, 39, 44, 51, 52, 54, 65, 68, 69, 75, 76, 78, 80, 85, 90, 91, 93, 96, 98, 102, 106, 109, 114, 119, 129, 130, 131, 132, 133, 136, 137, 139, 141, 144, 145, 149, 151, 152, 154, 155, 157, 158, 161, 166, 167, 173, 175, 177, 178, 183, 185, 189, 190, 196, 202, 206, 207, 209, 214, 217, 218, 225, 228, 231, 234, 239, 241, 244, 246, 249, 251, 257, 258, 269, 272, 274, 278, 279, 282, 284, 289, 290, 291, 292] 
 100  unique IDs found.
ID_TARGET : 34 with rows 2715


Function filter_data

In [26]:
def filtered_data(i,print_rows=False):
    """Filter the DataFrame by a specific ID_TARGET."""
    filtered_data = All_data[All_data['ID_TARGET'] == i]
    if filtered_data.empty:
        print("No data found for ID_TARGET:", i)
        return None
    else:
        if print_rows:
            print("ID_TARGET :",str(i),"with rows",len(filtered_data))
        return All_data[All_data['ID_TARGET'] == i]
    
# filtered_data(34)  # Example usage of the function

Data Manipulation

In [27]:
def columns_selection_loop(abs_data_set_sorted,train_data_x ,min_corr_x_to_y=0.25, max_corr_x_to_x=0.35,print_adopted_columns=False):
    """Select columns based on correlation threshold."""
    adopted_columns = []
    adopted_columns.append(abs_data_set_sorted.head(1).index[0])  # Add the highest abs correlated column

    for i in range(1, len(abs_data_set_sorted)):
        approval = True
        if abs_data_set_sorted.iloc[i] > min_corr_x_to_y and abs_data_set_sorted.index[i] not in adopted_columns:
            for j in range(len(adopted_columns)):
                if train_data_x[adopted_columns[j]].corr(train_data_x[abs_data_set_sorted.index[i]]) > max_corr_x_to_x:
                    approval = False
                    break
            if approval:
                adopted_columns.append(abs_data_set_sorted.index[i])
    if print_adopted_columns:
        print("Adopted columns:", adopted_columns, "\nwith a total of", len(adopted_columns), "columns")
    return adopted_columns

def columns_selection(x_train,y_train, min_corr_x_to_y=0.25, max_corr_x_to_x=0.35):
    corr_x_to_y = x_train.corrwith(y_train)
    abs_corr_x_to_y = corr_x_to_y.abs()
    abs_corr_x_to_y_sorted = abs_corr_x_to_y.sort_values(ascending=False)
    return columns_selection_loop(abs_corr_x_to_y_sorted,x_train, min_corr_x_to_y, max_corr_x_to_x)

Multiple Linear Regression Model

In [ ]:
# Multiple Linear Regression Model

def Multiple_linear_regression(x, y, print_summary=False):
    """
    Fit a linear regression model and return the model.
    
    Parameters:
    x (DataFrame): Feature matrix.
    y (Series): Target variable.
    
    Returns:
    LinearRegression: Fitted linear regression model and summary.
    """
    X = pd.DataFrame(x)  # Constuct X as a DataFrame of x
    y = pd.Series(y)  # Ensure y is a Series

    model = LinearRegression()
    model.fit(X, y)
    
    # Print the summary statistics using statsmodels
    if print_summary:    
        model_sm = sm.OLS(y, sm.add_constant(X)).fit()  # Fit the model using statsmodels
        print(model_sm.summary())

        # RSS and RSE
        RSS = lambda x: np.sum((y - model.predict(x)) ** 2)
        RSE = lambda x: np.sqrt(RSS(x) / (len(x) - 2))

        # Calculate RSS and RSE for the model
        rss_value = RSS(X)
        rse_value = RSE(X)

        print()
        print("RSS", rss_value,"RSE", rse_value)
    
    return model

def Multiple_linear_regression_vector(x,y,print_summary=False):
    """
    Fit a linear regression model and return the model.
    
    Parameters:
    x (DataFrame): Feature matrix.
    y (Series): Target variable.
    
    Returns:
    LinearRegression: Fitted linear regression model and summary.
    """
    X = pd.DataFrame(x)  # Constuct X as a DataFrame of x
    y = pd.Series(y)  # Ensure y is a Series

    model = LinearRegression()
    model.fit(X, y)
    model_vector = [model.intercept_]
    for i in range(len(X.columns)):
        model_vector.append(model.coef_[i])
    
    # Print the summary statistics using statsmodels
    if print_summary:
        model_sm = sm.OLS(y, sm.add_constant(X)).fit()  # Fit the model using statsmodels
        print(model_sm.summary())

        # RSS and RSE
        RSS = lambda x: np.sum((y - model.predict(x)) ** 2)
        RSE = lambda x: np.sqrt(RSS(x) / (len(x) - 2))

        # Calculate RSS and RSE for the model
        rss_value = RSS(X)
        rse_value = RSE(X)

        print()
        print("RSS", rss_value,"RSE", rse_value)
    
    model_vector = np.array(model_vector)
    return model_vector

In [29]:
def Regression_model(i,min_corr_x_to_y=0.25, max_corr_x_to_x=0.35):
    train_data = filtered_data(i)
    train_data_x = train_data.drop(columns=['ID_TARGET', 'RET_TARGET'])
    train_data_y = train_data['RET_TARGET']
    selected_train_data_x = train_data_x[columns_selection(train_data_x, train_data_y, min_corr_x_to_y, max_corr_x_to_x)]
    return Multiple_linear_regression(selected_train_data_x, train_data_y)

def Regression_model_vector(i,min_corr_x_to_y=0.25, max_corr_x_to_x=0.35):
    train_data = filtered_data(i)
    train_data_x = train_data.drop(columns=['ID_TARGET', 'RET_TARGET'])
    train_data_y = train_data['RET_TARGET']
    selected_train_data_x = train_data_x[columns_selection(train_data_x, train_data_y, min_corr_x_to_y, max_corr_x_to_x)]
    return Multiple_linear_regression_vector(selected_train_data_x, train_data_y)

In [30]:
Regression_model_dict = {str(i): Regression_model(i,0.2374,0.6) for i in unique_ids}
Regression_model_vector_dict = {str(i): Regression_model_vector(i,0.2374,0.6) for i in unique_ids}
Columns_selestion_dict = {str(i): columns_selection(filtered_data(i).drop(columns=['ID_TARGET', 'RET_TARGET']), filtered_data(i)['RET_TARGET'],0.2374,0.6) for i in unique_ids}

Import Test Data

In [31]:
x_test_path = root_path + "/X_test_Beg4ey3.csv"
x_test = pd.read_csv(x_test_path)
x_test.fillna(0, inplace=True)
x_test.head(20)
len(x_test)

114468

Model & Prediction

In [32]:
y_test = pd.DataFrame()
y_test['ID'] = x_test['ID']
RET_TARGET_test=[]

for i in range(len(x_test)):

    x_test_row = x_test.iloc[i]  # Select the first column as ID
    model_ID = str(int(x_test_row.loc['ID_TARGET']))  # Get the ID_TARGET value from the first row
    
    x_test_row_vector = [[1]]
    for ind in Columns_selestion_dict[model_ID]:
         x_test_row_vector.append([x_test_row.loc[ind]])
         
    x_test_row_vector = np.array(x_test_row_vector)
    
    Model_vector = Regression_model_vector_dict[model_ID].reshape(1,1+len(Columns_selestion_dict[model_ID]))

    result = Model_vector@x_test_row_vector
    
    if result >=0:
            result = 1
    else:
            result = -1
    
    RET_TARGET_test.append(result)

y_test['RET_TARGET'] = pd.Series(RET_TARGET_test)

Export result as CSV

In [ ]:
output_location ='C:/QRT 2025 Reconstruction of Liquid Asset Performance/Result'
y_test.to_csv(output_location +'/output.csv', index=False)